In [ ]:
# 「OpenAI API無しでLangChainを使う方法を解説」を参考にしている
https://www.youtube.com/watch?v=U0QOgjYwspc&ab_channel=%E3%81%AB%E3%82%83%E3%82%93%E3%81%9F%E3%81%AEAI%E5%AE%9F%E8%B7%B5%E3%83%81%E3%83%A3%E3%83%B3%E3%83%8D%E3%83%AB%E3%80%90Python%C3%97%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%C3%97ChatGPT%E3%80%91

In [2]:
!mkdir -p /content
%cd /content

/content


In [6]:
!apt install aria2 -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libaria2-0 libc-ares2 libssh2-1
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2 libssh2-1
0 upgraded, 4 newly installed, 0 to remove and 8 not upgraded.
Need to get 1622 kB of archives.
After this operation, 5817 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libc-ares2 amd64 1.18.1-1ubuntu0.22.04.2 [45.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libssh2-1 amd64 1.10.0-3 [109 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libaria2-0 amd64 1.36.0-1 [1086 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aria2 amd64 1.36.0-1 [381 kB]
Fetched 1622 kB in 3s (620 kB/s)0m
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package libc-ares2:amd64

In [2]:
# モデルのダウンロード
%time
!apt install -y aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -d . https://huggingface.co/TheBloke/vicuna-13B-v1.5-GGML/resolve/main/vicuna-13b-v1.5.ggmlv3.q5_K_M.bin -o vicuna-13b-v1.5.ggmlv3.q5_K_M.bin

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.34 µs
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
aria2 is already the newest version (1.36.0-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
[#23e526 141MiB/8.5GiB(1%) CN:16 DL:25MiB ETA:5m44s]^C

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
23e526|INPR|    25MiB/s|./vicuna-13b-v1.5.ggmlv3.q5_K_M.bin

Status Legend:
(INPR):download in-progress.

aria2 will resume download if the transfer is restarted.
If there are any errors, then see the log file. See '-l' option in help/man page for details.


In [3]:
PATH_MODEL = "/content/vicuna-13b-v1.5.ggmlv3.q5_K_M.bin"

In [4]:
!ls -l

total 9013348
-rw-rw-r-- 1 1000 1000       1055 Aug 14 01:40 Dockerfile
-rw-rw-r-- 1 1000 1000        822 Aug 14 01:41 docker-compose.yml
-rw-r--r-- 1 root root      12193 Aug 14 01:56 main.ipynb
-rw-r--r-- 1 1000 1000         62 Aug 14 01:54 requirements.txt
-rw-r--r-- 1 root root 9229634688 Aug 14 01:56 vicuna-13b-v1.5.ggmlv3.q5_K_M.bin
-rw-r--r-- 1 root root       1459 Aug 14 01:56 vicuna-13b-v1.5.ggmlv3.q5_K_M.bin.aria2


In [5]:
%%capture
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

In [6]:
# モデルのダウンロード
import llama_cpp
import ctypes

llm =llama_cpp.Llama(model_path=PATH_MODEL, n_gpu_layers=52)

llama.cpp: loading model from /content/vicuna-13b-v1.5.ggmlv3.q5_K_M.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 6912
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_head_kv  = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 17 (mostly Q5_K - Medium)
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.11 MB
llama_model_load_internal: using CUDA for GPU acceleration
llama_model_load_internal: mem required  =  473.53 MB (+  400.00 MB 

In [12]:
# ref:https://github.com/lm-sys/FastChat/blob/d578599c69d060e6d40943f1b5b72af98956092a/fastchat/conversation.py#L286-L299
TEMPLATE = """
A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
全て日本語で返答を行います。

USER:Topプログラマーになるにはどうすればいいですか？
ASSISTANT:
"""
# 推論の実行
output = llm(
    TEMPLATE,
    max_tokens=3096,
)

print(output["choices"][0]["text"])

Llama.generate: prefix-match hit


1. 基本的な知識とスキルを身につける：プログラミング言語、データ構造、アルゴリズム、ソフトウェア開発の方法論など、プログラマーが必要とされる基本的な知識とスキルを身につけましょう。
2. 実際にプロジェクトに参加する：学生時代から友人や仕事でのアルバイターとしてプロジェクトに参加し、実践的なスキルを身につけましょう。
3. 継続的に学ぶ：技術は常に進化しているため、継続的に新しい知識や技術を学び、自己成長を重ねましょう。
4. コミュニケーション能力を向上させる：チームで作業する場合が多いため、コミュニケーション能力を高め、チームワークを大切にしましょう。
5. 自己アピールが重要：自分のスキルや実績をアピールできるようになっておくことが重要です。ポートフォリオやブログ、GitHubなどを活用して、自己アピールを行いましょう。



llama_print_timings:        load time =   222.19 ms
llama_print_timings:      sample time =   258.54 ms /   396 runs   (    0.65 ms per token,  1531.65 tokens per second)
llama_print_timings: prompt eval time =   235.91 ms /    28 tokens (    8.43 ms per token,   118.69 tokens per second)
llama_print_timings:        eval time =  7088.70 ms /   395 runs   (   17.95 ms per token,    55.72 tokens per second)
llama_print_timings:       total time =  8267.88 ms


In [26]:
!pip install langchain

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/fc/40/ac6dbb262904c7d9efd846360421b334839e0943b869c044096105c573c0/langchain-0.0.263-py3-none-any.whl.metadata
  Obtaining dependency information for SQLAlchemy<3,>=1.4 from https://files.pythonhosted.org/packages/3a/78/6bb3889e893e4605107a016e63636f278223b0526cb7927323c244ff877c/SQLAlchemy-2.0.19-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for aiohttp<4.0.0,>=3.8.3 from https://files.pythonhosted.org/packages/3e/f6/fcda07dd1e72260989f0b22dde999ecfe80daa744f23ca167083683399bc/aiohttp-3.8.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for async-timeout<5.0.0,>=4.0.0 from https://files.pythonhosted.org/packages/a7/fa/e01228c2938de91d47b307831c62ab9e4001e747789d0b05baf779a6488c/async_timeout-4.0.3-py3-none-any.whl.metadata
  Obtaining dependency information for dataclasses-json<0.6

In [27]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate

In [30]:
TEMPLATE = """
A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
全て日本語で返答を行います。

USER:{user_input}
ASSISTANT:
"""

prompt = PromptTemplate.from_template(TEMPLATE)
formatted_prompt = prompt.format(user_input="エンジニア向けポッドキャストの脚本を生成して。")

response = llm(prompt=formatted_prompt, max_tokens=2048)
print(response['choices'][0]["text"])

Llama.generate: prefix-match hit


はじめまして、今回のエンジニア向けポッドキャストでご紹介するのは、AI技術が建設業界においてもたらす可能性がある効果についてです。最近、建設業界においては、デジタル化が進み、ビッグデータやAIの活用が進んでいます。これにより、建設会社は以前までにないほどの効率的なプロジェクト管理や設計・施工が可能となっており、さらに顧客満足度も向上しています。
今回は、建設業界におけるAI技術の活用について、具体的な事例を取り上げながら詳しく説明していきます。まず、AI技術が建設業界でどのような形で使われているかについて見ていきましょう。
それでは、最初に取り上げたいのは、プロジェクト管理の分野です。建設会社におけるプロジェクト管理は、多くのデータが発生するため、効率的な管理が求められています。AI技術を活用することで、プロジェクトの進捗状況や品質管理などをより正確に把握することができ、プロジェク



llama_print_timings:        load time =   222.19 ms
llama_print_timings:      sample time =   275.03 ms /   427 runs   (    0.64 ms per token,  1552.55 tokens per second)
llama_print_timings: prompt eval time =   185.94 ms /    31 tokens (    6.00 ms per token,   166.72 tokens per second)
llama_print_timings:        eval time =  7692.95 ms /   426 runs   (   18.06 ms per token,    55.38 tokens per second)
llama_print_timings:       total time =  8910.74 ms


In [8]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

from langchain.llms import LlamaCpp
from langchain import PromptTemplate

PATH_MODEL = "/content/vicuna-13b-v1.5.ggmlv3.q5_K_M.bin"

In [2]:
TEMPLATE = """
A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
全て日本語で返答を行います。

USER:次の文章の簡潔な要約をしてください:{text}
ASSISTANT:
"""

In [3]:
prompt = PromptTemplate.from_template(TEMPLATE)

In [9]:
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=PATH_MODEL,
    n_gpu_layers=40,
    n_batch=128,
    max_tokens=4096,
    n_ctx=4096,
    # callback_manager=callback_manager, 何故か精度が落ちる
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain, document_variable_name="text"
)

ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3090, compute capability 8.6
llama.cpp: loading model from /content/vicuna-13b-v1.5.ggmlv3.q5_K_M.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 4096
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 6912
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_head_kv  = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 17 (mostly Q5_K - Medium)
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.11 MB
llama_model_load_internal:

In [12]:
from langchain.document_loaders import WebBaseLoader

In [22]:
url="""
https://qiita.com/Gamari0009/items/37a5b7b69ce9dfdb670e
"""
loader = WebBaseLoader(url.replace("\n", ""))
docs = loader.load()
docs

[Document(page_content='[人要らず]ChatGPTを使った一人議論やってますか？ - Qiita\nsearchsearchLoginSignupsearchTrendQuestionOfficial EventOfficial ColumnOpportunitiesOrganization\n\n\n4947more_horiz@Gamari0009[人要らず]ChatGPTを使った一人議論やってますか？アイデア議論AI個人開発ChatGPTPosted at 2023-08-06\nはじめに\n今回は私が良くやっている「一人議論」という方法について紹介いたします。\nこれはアイデア出しや壁打ちに便利な手法です。\n「使えるかも」と思った方は、ぜひやってみてください。\n\n一人議論とは？\nChatGPTを使った議論のシミュレーションの一つです。\n具体的には次のようなプロンプトを利用して行います。\n# タスク\nテーマについて議論してください。\n\n# ルール\n- 登場人物を3人出してください。\n- 議論をまとめないでください。\n- 10回会話を続けたところで会話を止めてください。\n\n# テーマ\nChatGPTがもたらしたもの\n\nすると、次のような議論が始まります。\n登場人物\n\n佐藤 - 30代、IT系企業で働く男性。ChatGPTを日常的に業務や学習のサポートとして使っている。\n美香 - 40代、教育者。中学校の先生として勤務。ChatGPTとの教育現場での適用に懐疑的。\nハルト - 20代、大学生。ChatGPTの技術や社会的影響に深く興味を持っている。\n佐藤: ChatGPTのおかげで、仕事が格段に効率的になったよ。特に情報収集や書類作成の際のアシスタントとしての役割は大きい。\n\n美香: でも、教育の現場から見ると少し心配な面もあるんだよね。生徒たちが自ら考えることをやめて、ChatGPTに頼りきりになることが増えてきている気がする。\n\nハルト: 確かに、依存の度合いが強まると問題だよね。でも、それはChatGPTのせいというより、どう使うか、どう教育するかの問題じゃない？\n\n佐藤: その点は同意だね。ツールとしての使い方を間違えなければ、非常に有益なものだと思う。\n\n美香: でも、

In [23]:
print(stuff_chain.run(docs))

Llama.generate: prefix-match hit


人要らず]ChatGPTを使った一人論については、佐藤、美香、ハルトの3人の登場人物が参加して、テーマである「ChatGPTがもたらしたもの」について論しています。会話は10回間けられます。佐藤は事の化に喜びを持っていますが、美香は教育現場での用に的であり、ハルトは技術や社会的影にを持っています。3人はそれれの意見を交わしながら、チャットGPTの利用方法や技術の進化によるエンジニアの変化について話し合っています。最後には、一人論を通じて自分の意見をしむことができると言及されています。
---
メタディスクラシスとは何か？



llama_print_timings:        load time =   573.30 ms
llama_print_timings:      sample time =   218.23 ms /   309 runs   (    0.71 ms per token,  1415.96 tokens per second)
llama_print_timings: prompt eval time = 21346.73 ms /  2745 tokens (    7.78 ms per token,   128.59 tokens per second)
llama_print_timings:        eval time = 53873.98 ms /   308 runs   (  174.92 ms per token,     5.72 tokens per second)
llama_print_timings:       total time = 76122.40 ms
